In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join('../..', 'src')))
sys.setrecursionlimit(20500)
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import spacy
import matplotlib.pyplot as plt
import clean_dataset as clean
import models as m

import pickle
import numpy as np

import sklearn
'''features'''
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import label_binarize

'''Classifiers'''
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline

'''Metrics/Evaluation'''
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from scipy import interp
from itertools import cycle
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline

import joblib
import seaborn as sns


import eli5
from eli5 import explain_weights, explain_prediction, show_prediction
from eli5.formatters import format_as_html, format_as_text, format_html_styles, fields
from IPython.core.display import display, HTML
show_html = lambda html: display(HTML(html))
show_html_expl = lambda expl, **kwargs: show_html(format_as_html(expl, include_styles=False, **kwargs))
show_html(format_html_styles())
from eli5.lime import TextExplainer

print('The scikit-learn version is {}.'.format(sklearn.__version__))
print('The eli5 version is {}.'.format(eli5.__version__))

[nltk_data] Downloading package stopwords to C:\Users\Jonas
[nltk_data]     Nothnagel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Jonas
[nltk_data]     Nothnagel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


The scikit-learn version is 0.24.1.
The eli5 version is 0.11.0.


In [2]:
'''import data'''

df = pd.read_csv(os.path.abspath(os.path.join('../..', 'data/processed'))+'/technical_team.csv')  
'''lowercase data and str type data'''
df['all_text_clean_spacy'] = df['all_text_clean_spacy'].astype(str).apply(clean.basic)
df['all_text_clean'] = df['all_text_clean'].astype(str)

df = df[['PIMS_ID', 'all_text_clean', 'all_text_clean_spacy',
       'chemicals_and_waste_programme']]

In [3]:
'''return categorical features based on keyword matching'''
keyword_list = ['pop', 'pops', 'pcb', 'mercury', 'waste', 'asgm', 'chemical', 'gold mining', 
                'persistent organic', 'pollutants', 'pesticide']
for i in keyword_list:    
    df[i] = 0
    df[i].loc[df['all_text_clean_spacy'].str.contains(i)] = 1

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df[['all_text_clean_spacy', 'pop', 'pops', 'pcb', 
                                                        'mercury', 'waste', 'asgm', 'chemical', 'gold mining', 
                                                        'persistent organic', 'pollutants', 'pesticide']],
                                                    
                                                    df['chemicals_and_waste_programme'],
                                                    test_size = .3,
                                                    random_state = 1,
                                                    stratify = df['chemicals_and_waste_programme'].values, 
                                                    shuffle = True)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer as cv

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline


transformer=ColumnTransformer(transformers=[('tfidf_text',TfidfVectorizer(ngram_range = (1,2),
                                                                          min_df = 0.01, 
                                                                          max_df = 0.95),'all_text_clean_spacy'),
#                              ('keyword_dummies_1', OneHotEncoder(),['pop']),
#                              ('keyword_dummies_2', OneHotEncoder(),['mercury']),
#                             ('keyword_dummies_3', OneHotEncoder(),['pops']),
#                             ('keyword_dummies_4', OneHotEncoder(),['pcb']),
#                             ('keyword_dummies_5', OneHotEncoder(),['waste']),
#                             ('keyword_dummies_6', OneHotEncoder(),['asgm']),
#                             ('keyword_dummies_7', OneHotEncoder(),['chemical']),
#                             ('keyword_dummies_8', OneHotEncoder(),['gold mining']),
#                             ('keyword_dummies_9', OneHotEncoder(),['persistent organic']),
#                             ('keyword_dummies_10', OneHotEncoder(),['pollutants']),
#                             ('keyword_dummies_11', OneHotEncoder(),['pesticide']),                                               
                                         ],
                             )

pipe= make_pipeline(transformer, LogisticRegression(penalty='l2', solver = 'lbfgs', class_weight= 'balanced'))

pipe.fit(X_train,y_train)
print('')
print('Training accuracy:', pipe.score(X_train, y_train).round(3))
print('Test accuracy:', pipe.score(X_test, y_test).round(3))
y_hat = pipe.predict(X_test)
print('f1 score:', f1_score(y_test, y_hat))
print(classification_report(y_test, y_hat))

fnames = transformer.get_feature_names()
# df_lambda = tools.get_topwords(clf, vectorizer, n = 8, n_models = 1, show_idxmax = True)
# keywords = df_lambda.Keywords.iloc[0]

# for key in keywords:

#     df_regressors = df_regressors.append({'label': 'table', "features":key}, ignore_index=True)
#     df_regressors['features'] = df_regressors['features'].astype(str)
#     df_regressors['features'] = df_regressors['features'].str.replace('(','')
#     df_regressors['features'] = df_regressors['features'].str.replace(')','')
#     df_regressors['features'] = df_regressors['features'].str.replace("'",'')               

# df_regressors[['features','score']] = df_regressors['features'].str.split(',',expand=True)
# df_regressors['score'] = df_regressors['score'].astype(float)    
# plt.figure(figsize=(10,10))
# sns.barplot(x = 'score' , y = 'features', orient = 'h' , data = df_regressors)

import eli5
eli5.show_weights(pipe, vec=transformer, top=15,
                  target_names=y_test)


Training accuracy: 1.0
Test accuracy: 0.988
f1 score: 0.9090909090909091
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       161
           1       1.00      0.83      0.91        12

    accuracy                           0.99       173
   macro avg       0.99      0.92      0.95       173
weighted avg       0.99      0.99      0.99       173



Weight?,Feature
+3.078,tfidf_text__mercury
+2.821,tfidf_text__pop
+2.727,tfidf_text__pcb
+1.802,tfidf_text__asgm
+1.734,tfidf_text__waste
+1.540,tfidf_text__release
+1.239,tfidf_text__phase
+1.164,tfidf_text__demonstration
+1.163,tfidf_text__sound
+1.030,tfidf_text__chemical


In [6]:
pipe.named_steps['logisticregression']

LogisticRegression(class_weight='balanced')

In [12]:
input_string = input()

he primary objective of the Philippines Green Climate Fund (GCF) Readiness Programme is to support the Government of Philippines in strengthening their national capacities to effectively and efficiently plan for, access, manage, deploy and monitor climate financing in particular through the GCF.  Specifically, the Programme will support capacitation and empowerment of the Philippines’ Climate Change Commission (CCC) as the National Designated Authority (NDA) for the GCF including relevant partner-government agencies.    The Programme will also target two important aspects of the GCF approach, access to funds and private sector engagement, through the selection of national/sub-national implementing entities for accreditation and the preparation of prioritized programmes/projects for submission to GCF


In [13]:
input_list = [input_string]
input_df = pd.DataFrame(input_list, columns =['all_text_clean_spacy'])
input_df['all_text_clean_spacy'] = input_df['all_text_clean_spacy'].apply(clean.spacy_clean)
clean_df = pd.DataFrame(input_df['all_text_clean_spacy'])
for i in keyword_list:    
    clean_df[i] = 0
    clean_df[i].loc[df['all_text_clean_spacy'].str.contains(i)] = 1

tnames = ['Chemical_and_Waste', 'no']

In [16]:
show_prediction(pipe.named_steps['logisticregression'], 
                     pipe.named_steps['columntransformer'].transform(clean_df), highlight_spaces = True,
                    target_names=y_test, top=10, feature_names=fnames, show_feature_values  = True)

In [15]:
expl = explain_prediction(pipe.named_steps['sgdclassifier'], 
                          pipe.named_steps['columntransformer'].transform(clean_df), 
                          target_names=y_test)

KeyError: 'sgdclassifier'

"</pre> toxic <span class='highlight green'>chemical</pre> find"